In [1]:
# load data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", reshape=False)

x_train, y_train = mnist.train.images, mnist.train.labels
x_validation, y_validation = mnist.validation.images, mnist.validation.labels
x_test, y_test = mnist.test.images, mnist.test.labels

assert(len(x_train)==len(y_train))
assert(len(x_validation)==len(y_validation))
assert(len(x_test)==len(y_test))

print()
print("Images Shape: {}".format(x_train[0].shape))
print()
print("Training Set:   {} samples".format(len(x_train)))
print("Validation Set: {} samples".format(len(x_validation)))
print("Testing Set:    {} samples".format(len(x_test)))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.

Images Shape: (28, 28, 1)

Training Set:   55000 samples
Validation Set: 5000 samples
Testing Set:    10000 samples


In [2]:
# pad images with 0s 
import numpy as np 
x_train = np.pad(x_train, ((0,0), (2,2), (2,2), (0,0)), 'constant')
x_validation = np.pad(x_validation, ((0,0), (2,2), (2,2), (0,0)), 'constant')
x_test = np.pad(x_test, ((0,0), (2,2), (2,2), (0,0)), 'constant')

print("Updated Image Shape: {}".format(x_train[0].shape))

Updated Image Shape: (32, 32, 1)
